In [1]:
import os
from pathlib import Path
import sys
import time
from functools import partial
from tqdm import tqdm
import copy
import  random

from pprint import pprint

pwd = Path(os.getcwd())
sys.path.append(str(pwd.parent.parent / "gym-checkers-for-thai"))

In [2]:
from checkers.agents.baselines import play_a_game, RandomPlayer
from checkers.game import Checkers
from checkers.agents import Player
from checkers.agents.alpha_beta import MinimaxPlayer, first_order_adv

from player import DeepLearningPlayer
# from model.small_model import GDQL as GDQL
from model.medium_model import GDQL_m as GDQL

import mlflow
import mlflow.pytorch

import matplotlib.pyplot as plt
import seaborn as sns

import torch

import numpy as np

In [3]:
MINIMAX_SEARCH_DEPTH = 2
WEIGHT_FOLDER = pwd / "weights" / f"vs_depth_{MINIMAX_SEARCH_DEPTH}"

N_EPISODES = 100
N_MATCHES_PER_EPS = 50

REWARD_DISCOUNT_FACTOR = 0.95

EPSILON = 0.9
EPSILON_DECAY_FACTOR = 0.999
EPSILON_MIN = 0.33

WIN_REWARD = 100
LOSE_REWARD = -40
DRAW_REWARD = -20

BATCH_SIZE = 256

TARGET_UPDATE = 4 # update target network every TARGET_UPDATE episodes
LEARNING_RATE = 1e-3

In [4]:
try:
    mlflow.end_run()
except:
    pass

In [5]:
mlflow.set_experiment("DQL with gredient descent (medium model)")
mlflow.start_run()
mlflow.log_param("MINIMAX_SEARCH_DEPTH", MINIMAX_SEARCH_DEPTH)
mlflow.log_param("N_EPISODES", N_EPISODES)
mlflow.log_param("N_MATCHES_PER_EPS", N_MATCHES_PER_EPS)
mlflow.log_param("REWARD_DISCOUNT_FACTOR", REWARD_DISCOUNT_FACTOR)
mlflow.log_param("EPSILON", EPSILON)
mlflow.log_param("EPSILON_DECAY_FACTOR", EPSILON_DECAY_FACTOR)
mlflow.log_param("EPSILON_MIN", EPSILON_MIN)
mlflow.log_param("BATCH_SIZE", BATCH_SIZE)
mlflow.log_param("TARGET_UPDATE", TARGET_UPDATE)
mlflow.log_param("LEARNING_RATE", LEARNING_RATE)
mlflow.log_param("WIN_REWARD", WIN_REWARD)
mlflow.log_param("LOSE_REWARD", LOSE_REWARD)
mlflow.log_param("DRAW_REWARD", DRAW_REWARD)

-20

In [6]:
# Create the folder if it doesn't exist
WEIGHT_FOLDER.mkdir(parents=True, exist_ok=True)

In [7]:
online_model = GDQL(lr=LEARNING_RATE)
target_model = GDQL(lr=LEARNING_RATE)
try:
    online_model.load_state_dict(torch.load(WEIGHT_FOLDER / "online_model.pth"))
    target_model.load_state_dict(torch.load(WEIGHT_FOLDER / "target_model.pth"))
except FileNotFoundError:
    print("No weights found, starting from scratch")
except RuntimeError:
    print("Weights are corrupted, starting from scratch")

max_win_rate = 0

for episode in range(N_EPISODES):
    stime = time.time()
    n_wins, n_losses, n_draws = 0, 0, 0
    mean_loss = 0
    DeepLearningPlayer.experience.clear()


    looper = tqdm(range(N_MATCHES_PER_EPS), unit="matches", leave=True, desc=f"Episode {episode+1}")
    for i in looper:
        ch = Checkers()

        black_player = DeepLearningPlayer('black',
                                model=online_model,
                                epsilon=EPSILON,
                                epsilon_decay=EPSILON_DECAY_FACTOR,
                                epsilon_min=EPSILON_MIN,
                                win_reward=WIN_REWARD,
                                lose_reward=LOSE_REWARD,
                                draw_reward=DRAW_REWARD,)

        if MINIMAX_SEARCH_DEPTH == 0:
            # Random player function
            white_player = RandomPlayer('white', seed=i)
        else:
            # Minimax player function
            white_player = MinimaxPlayer('white', 
                                        partial(first_order_adv, 'white', 86, 54.5, 87, 26),
                                        search_depth=MINIMAX_SEARCH_DEPTH)
        
        # push into environment
        winner = play_a_game(ch, black_player.next_move, white_player.next_move, 100, is_show_detail=False)
        if winner == 'black':
            n_wins += 1
            black_player.set_win()
        elif winner == 'white':
            n_losses += 1
            black_player.set_lose()
        else:
            n_draws += 1

        if len(DeepLearningPlayer.experience) > BATCH_SIZE:
            batch_states = random.sample(DeepLearningPlayer.experience, BATCH_SIZE)
            
            # find target, online Q values and compute loss
            loss = 0
            for batch_idx, (state, action, reward, next_state) in enumerate(batch_states):
                online_model.train()
                target_model.eval()

                # find target Q
                if next_state is not None:
                    max_next_state_value = -np.inf
                    ch.restore_state(next_state)
                    available_actions = ch.legal_moves()
                    for available_action in available_actions:
                        model_input = target_model.board2input(next_state[0], 'black', available_action)
                        next_state_value = target_model(model_input)
                        max_next_state_value = max(max_next_state_value, next_state_value)
                    target_q = reward + max_next_state_value * REWARD_DISCOUNT_FACTOR
                else:
                    target_q = reward

                # find online Q
                model_input = online_model.board2input(state[0], 'black', action)
                online_q = online_model(model_input)

                loss += (online_q - target_q) ** 2
            loss /= BATCH_SIZE
            mean_loss += loss.item()
            looper.set_postfix(loss=loss.item(),
                               win_rate=n_wins / (i+1),)

            # compute loss
            online_model.optimizer.zero_grad()
            loss.backward()
            online_model.optimizer.step()

    if episode % TARGET_UPDATE == 0:
        target_model.load_state_dict(online_model.state_dict())
        print("\tTarget model updated")
    print(f"\tWins: {n_wins}, Losses: {n_losses}, Draws: {n_draws}")

    mlflow.log_metric("runl time", time.time() - stime, step=episode)
    mlflow.log_metric("win rate", n_wins / N_MATCHES_PER_EPS, step=episode)
    mlflow.log_metric("draw rate", n_draws / N_MATCHES_PER_EPS, step=episode)
    mlflow.log_metric("mean of mse loss", mean_loss / N_MATCHES_PER_EPS, step=episode)
    if n_wins / N_MATCHES_PER_EPS > max_win_rate:
        max_win_rate = n_wins / N_MATCHES_PER_EPS
        torch.save(online_model.state_dict(), WEIGHT_FOLDER / "online_model.pth")
        torch.save(target_model.state_dict(), WEIGHT_FOLDER / "target_model.pth")
        print(f"\tNew max win rate: {max_win_rate}")
        mlflow.pytorch.log_model(online_model, "models")
        mlflow.log_artifact(WEIGHT_FOLDER / "online_model.pth")
        mlflow.log_artifact(WEIGHT_FOLDER / "target_model.pth")

Weights are corrupted, starting from scratch


Episode 1: 100%|██████████| 50/50 [05:47<00:00,  6.95s/matches, loss=44.6, win_rate=0]


	Target model updated
	Wins: 0, Losses: 50, Draws: 0


Episode 2: 100%|██████████| 50/50 [05:51<00:00,  7.04s/matches, loss=13.4, win_rate=0]


	Wins: 0, Losses: 50, Draws: 0


...

	Wins: 0, Losses: 50, Draws: 0


Episode 99: 100%|██████████| 50/50 [05:17<00:00,  6.36s/matches, loss=8.29, win_rate=0]


	Wins: 0, Losses: 50, Draws: 0


Episode 100:  74%|███████▍  | 37/50 [04:51<01:42,  7.88s/matches, loss=8.77, win_rate=0]


KeyboardInterrupt: 

In [ ]:
mlflow.end_rlun()

In [ ]:
ch = Checkers()

black_player = DeepLearningPlayer('black',
                                model=online_model,
                                epsilon=EPSILON,
                                epsilon_decay=EPSILON_DECAY_FACTOR,
                                epsilon_min=EPSILON_MIN,)
# Random player function
white_player = RandomPlayer('white', seed=i)
        
# push into environment
winner = play_a_game(ch, black_player.next_move, white_player.next_move, 100, is_show_detail=True)

_b_b_b_b
b_b_b_b_
_._._._.
._._._._
_._._._.
._._._._
_w_w_w_w
w_w_w_w_
0 turn: black last_moved_piece: None
7 legal moves [(4, 8), (5, 8), (5, 9), (6, 9), (6, 10), (7, 10), (7, 11)]
black moved 6, 10

_b_b_b_b
b_b_._b_
_._._b_.
._._._._
_._._._.
._._._._
_w_w_w_w
w_w_w_w_
1 turn: white last_moved_piece: None
7 legal moves [(24, 21), (24, 20), (25, 22), (25, 21), (26, 23), (26, 22), (27, 23)]
white moved 25, 22

_b_b_b_b
b_b_._b_
_._._b_.
._._._._
_._._._.
._._w_._
_w_._w_w
w_w_w_w_
2 turn: black last_moved_piece: None
8 legal moves [(1, 6), (2, 6), (4, 8), (5, 8), (5, 9), (7, 11), (10, 14), (10, 15)]
black moved 10, 15

_b_b_b_b
b_b_._b_
_._._._.
._._._b_
_._._._.
._._w_._
_w_._w_w
w_w_w_w_
3 turn: white last_moved_piece: None
8 legal moves [(22, 18), (22, 17), (24, 21), (24, 20), (26, 23), (27, 23), (29, 25), (30, 25)]
white moved 27, 23

_b_b_b_b
b_b_._b_
_._._._.
._._._b_
_._._._.
._._w_w_
_w_._w_.
w_w_w_w_
4 turn: black last_moved_piece: None
9 legal moves [(1, 6), (2, 6), (4, 8),